In [2]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('NPPE1_Preprocessing1.csv')
df.head()

,CRIM,ZN,INDUS,POLINDEX,RM,AGE,DIS,HIGHWAYCOUNT,TAX,PTRATIO,IMM,BPL,PRICE,RIVERSIDE
0,1.026769,1.429034,7.851300,1.134216,6.0,42.0,5.251911,5,279.201277,20.689586,398.811960,10.461456,22.991633,NO
1,0.848089,0.255543,6.263434,1.245993,7.0,63.0,4.305546,8,307.444529,17.465398,377.153649,11.619690,24.551055,NO
2,10.925905,0.441022,18.322960,2.824833,8.0,-2.0,2.409495,25,666.492973,20.351601,387.061355,19.366070,15.875346,NO
3,0.559027,1.041175,11.114920,0.794952,6.0,9.0,6.898669,4,305.514181,19.787314,391.778647,6.206820,23.007756,NO
4,0.905063,81.167963,3.673369,1.029030,8.0,20.0,10.246463,1,315.913960,17.360439,395.833166,10.827105,21.503177,NO


In [4]:
averagePrice = df['PRICE'].mean()
print(averagePrice)

24.355923220694248


In [5]:
houses_with_more_than_5_bedrooms = df[df['RM'] >= 5]
print(houses_with_more_than_5_bedrooms.count())

CRIM            3953
ZN              3953
INDUS           3953
POLINDEX        3953
RM              3953
AGE             3953
DIS             3953
HIGHWAYCOUNT    3953
TAX             3953
PTRATIO         3953
IMM             3953
BPL             3953
PRICE           3953
RIVERSIDE       3953
dtype: int64


In [6]:
top_10_most_expensive_houses = df.nlargest(10, 'PRICE')
print(top_10_most_expensive_houses['PRICE'].mean())

52.36590175716407


In [7]:
# What is the total number of missing or unknown values in the number of rooms feature?
missing_rooms = df.where(df['RM'] <= 0).count()
print(missing_rooms)

CRIM            40
ZN              40
INDUS           40
POLINDEX        40
RM              40
AGE             40
DIS             40
HIGHWAYCOUNT    40
TAX             40
PTRATIO         40
IMM             40
BPL             40
PRICE           40
RIVERSIDE       40
dtype: int64


In [8]:
# What is the total number of missing or unknown values in the age feature?
missing_age = df.where(df['AGE'] <= 0).count()
print(missing_age)

CRIM            50
ZN              50
INDUS           50
POLINDEX        50
RM              50
AGE             50
DIS             50
HIGHWAYCOUNT    50
TAX             50
PTRATIO         50
IMM             50
BPL             50
PRICE           50
RIVERSIDE       50
dtype: int64


In [9]:
# What is the total number of missing or unknown values in the RIVERSIDE feature?
missing_riverside = df.where(df['RIVERSIDE'] == "UNKNOWN").count()
print(missing_riverside)

CRIM            88
ZN              88
INDUS           88
POLINDEX        88
RM              88
AGE             88
DIS             88
HIGHWAYCOUNT    88
TAX             88
PTRATIO         88
IMM             88
BPL             88
PRICE           88
RIVERSIDE       88
dtype: int64


In [10]:
# How many houses are on riverside and were built within the last 50 years (i.e. a house 50 years old or younger)? For this question, ignore the rows that have missing values in either riverside feature or age feature.
riverside_and_young = df[(df['RIVERSIDE'] == "YES") & (df['AGE'] <= 50) & (df['AGE'] > 0)]
print(riverside_and_young.count())

CRIM            44
ZN              44
INDUS           44
POLINDEX        44
RM              44
AGE             44
DIS             44
HIGHWAYCOUNT    44
TAX             44
PTRATIO         44
IMM             44
BPL             44
PRICE           44
RIVERSIDE       44
dtype: int64


In [11]:
# How many houses are near to exactly 6, 7 or 8 highways (all three inclusive)?
near_highways = df[(df['HIGHWAYCOUNT'] >= 6) & (df['HIGHWAYCOUNT'] <= 8)]
print(near_highways.count())

CRIM            1211
ZN              1211
INDUS           1211
POLINDEX        1211
RM              1211
AGE             1211
DIS             1211
HIGHWAYCOUNT    1211
TAX             1211
PTRATIO         1211
IMM             1211
BPL             1211
PRICE           1211
RIVERSIDE       1211
dtype: int64


In [12]:
# Create a column 'CATEGORY' and divide the houses in categories as following
# Category 1: House price < 10.0
# Category 2: House price >= 10.0 and < 20.0
# Category 3: House price >= 20.0 and < 30.0
# Category 4: House price >= 30.0 and < 40.0
# Category 5: House price >= 40.0
# Which category has the highest number of records?

def categorize(price):
    if price < 10:
        return 1
    elif price >= 10 and price < 20:
        return 2
    elif price >= 20 and price < 30:
        return 3
    elif price >= 30 and price < 40:
        return 4
    else:
        return 5
    
df['CATEGORY'] = df['PRICE'].apply(categorize)
category_counts = df['CATEGORY'].value_counts()
print(category_counts)

CATEGORY
3    2028
2    1158
4     503
5     268
1      43
Name: count, dtype: int64


In [13]:
# Drop all the rows that have missing values in any of the features where the missing value is represented by -1. or the string "UNKNOWN".
df = df.dropna()
df = df[(df['RM'] != -1) & (df['AGE'] != -1) & (df['HIGHWAYCOUNT'] != -1) & (df['RIVERSIDE'] != "UNKNOWN")]
print(df.count())



CRIM            3872
ZN              3872
INDUS           3872
POLINDEX        3872
RM              3872
AGE             3872
DIS             3872
HIGHWAYCOUNT    3872
TAX             3872
PTRATIO         3872
IMM             3872
BPL             3872
PRICE           3872
RIVERSIDE       3872
CATEGORY        3872
dtype: int64


In [14]:
X_train, X_test = train_test_split(df, test_size=0.3)
pipe1=ColumnTransformer([('scale1',MinMaxScaler(),[0,1]),
                            ('scale2',StandardScaler(),[2]),
                            ('scale3',MinMaxScaler(),[3]),
                            ('impute1',SimpleImputer(strategy='median'),[4]),
                            ('impute2',SimpleImputer(strategy='mean'),[5]),
                            ('scale6',MinMaxScaler(),[6,7,8,9,10,11]),
                            ('impute3',SimpleImputer(strategy='most_frequent'),[12]),
                        ])
pipe2=ColumnTransformer([('scale7',MinMaxScaler(),[4,5]),('encode1',OneHotEncoder(),[12])],remainder="passthrough")
pipe3=Pipeline([("ct1",pipe1),("ct2",pipe2)])
pipe3.fit_transform(X_train).shape


(2710, 2722)